# Analysis Template

A template with the tools to get started pulling data from a local db.

In [1]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
import sqlite3


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../../data/perps.db")


In [3]:
df_trades = pd.read_sql_query('SELECT * FROM trades', con)
df_trades.head(5)

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed
0,0x0000709c04c4a1f048371ca2e2641740b161f24db3da...,0x02bee0a17903cc1f2f48b4514ec23aae547fe4a3,1678414645,OP,sOPPERP,DelayedOffchain,2.213579,358.166389,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x654,17.112466,-34.997998,-4551.442300,0.000000,1
1,0x000080a930a760cdbe740044a11622e717257fbe7383...,0xbf975b1548f793f3bcc2ce84dcf8e78ece793914,1678799771,OP,sOPPERP,DelayedOffchain,2.810356,43622.283865,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x4f7,171.471797,23634.822642,-40201.742667,0.000000,1
2,0x000092e1e5d70976b281207016b94f03d7b5c7eca12c...,0x6662304e3198c8d54530fbdf02a869fa84537639,1678622207,OP,sOPPERP,DelayedOffchain,1.883386,8878.590606,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x70a,14.535012,0.000000,-6655.571600,-117749.887028,0
3,0x0000ed0354c8751d24a076aa665c6fa2ec8ba76f9e17...,0xad63e4d4be2229b080d20311c1402a2e45cf7e75,1678606893,sETH,sETHPERP,DelayedOffchain,1471.509927,25740.544182,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x13cd,87.150445,1352.612980,-59.000000,210.000000,0
4,0x00014ff5c5e5f57960a3ae7e612d19097eb9b1b36f52...,0x6b879c078ea4cc27bbdb486fcaa4fa27e6cbe97a,1678831321,FLOW,sFLOWPERP,DelayedOffchain,1.028709,123.782635,0x27665271210acff4fab08ad9bb657e91866471f0-0x2a,3.206586,0.000000,1466.140400,1466.140400,0
